# 단일 ETF, 시장 데이터를 이용한 이벤트 스터디

In [261]:
import pandas as pd
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# 주가 데이터 로드 (예: CSV 파일에서)
df = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/ESGU.csv', parse_dates=['Date'])
df.set_index('Date', inplace=True)

# 이벤트 날짜와 이벤트 윈도우, 추정 윈도우 설정
event_date = pd.Timestamp('2017-01-20')
estimation_window = 30  # 추정 기간: 이벤트 이전 30일
event_window = 10       # 이벤트 기간: 이벤트 전후 10일

# 이벤트 윈도우와 추정 윈도우 설정
estimation_start = event_date - pd.Timedelta(days=estimation_window + event_window)
estimation_end = event_date - pd.Timedelta(days=1)
event_start = event_date - pd.Timedelta(days=event_window)
event_end = event_date + pd.Timedelta(days=event_window)




In [262]:
df1 = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/ESGU.csv', parse_dates=['Date'])
df1.set_index('Date', inplace=True)

In [263]:
df2 = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/SPY.csv', parse_dates=['Date'])
df2.set_index('Date', inplace=True)

In [264]:
df3 = pd.concat([df1['Close'], df2['Close']],axis=1)

In [265]:
df3.columns = [['Stock_Price', 'Market_Price']]

In [266]:
# 추정 윈도우 데이터 추출
estimation_data = df3.loc[estimation_start:estimation_end]
event_data = df3.loc[event_start:event_end]

In [267]:
import numpy as np
event_data["Stock_Return"] = np.log(event_data["Stock_Price"] / event_data["Stock_Price"].shift(1))
event_data["Market_Return"] = np.log(event_data["Market_Price"] / event_data["Market_Price"].shift(1))

In [268]:

# 시장 수익률과 개별 주식 수익률 계산
estimation_data['Market_Return'] = estimation_data['Market_Price'].pct_change()
estimation_data['Stock_Return'] = estimation_data['Stock_Price'].pct_change()

# 회귀 분석을 통한 알파와 베타 추정
X = sm.add_constant(estimation_data['Market_Return'].dropna())
y = estimation_data['Stock_Return'].dropna()
model = sm.OLS(y, X).fit()
alpha, beta = model.params


In [269]:
event_data['Expected_Return'] = (alpha + beta * event_data['Market_Return'])
event_data

,Stock_Price,Market_Price,Stock_Return,Market_Return,Expected_Return
Date,,,,,
2017-01-10,44.065697,198.357468,NaN,NaN,NaN
2017-01-11,44.065697,198.918045,0.000000,0.002822,0.002711
2017-01-12,44.065697,198.418732,0.000000,-0.002513,-0.000574
2017-01-13,44.065697,198.874237,0.000000,0.002293,0.002385
2017-01-17,43.897419,198.173508,-0.003826,-0.003530,-0.001200
2017-01-18,43.897419,198.611496,0.000000,0.002208,0.002333
2017-01-19,43.897419,197.875732,0.000000,-0.003711,-0.001312
2017-01-20,43.897419,198.602737,0.000000,0.003667,0.003232
2017-01-23,43.897419,198.085953,0.000000,-0.002605,-0.000631


In [270]:
a = event_data['Stock_Return'].values
b = event_data['Expected_Return'].values

In [271]:
event_data['Abnormal_Return'] = a-b
event_data['CAR'] = event_data['Abnormal_Return'].cumsum()


event_data[['Stock_Return', 'Expected_Return', 'Abnormal_Return', 'CAR']]

,Stock_Return,Expected_Return,Abnormal_Return,CAR
Date,,,,
2017-01-10,NaN,NaN,NaN,NaN
2017-01-11,0.000000,0.002711,-0.002711,-0.002711
2017-01-12,0.000000,-0.000574,0.000574,-0.002137
2017-01-13,0.000000,0.002385,-0.002385,-0.004522
2017-01-17,-0.003826,-0.001200,-0.002626,-0.007149
2017-01-18,0.000000,0.002333,-0.002333,-0.009481
2017-01-19,0.000000,-0.001312,0.001312,-0.008170
2017-01-20,0.000000,0.003232,-0.003232,-0.011401
2017-01-23,0.000000,-0.000631,0.000631,-0.010770


# 다중 ETF, 시장 데이터를 이용한 이벤트 스터디

In [272]:
import pandas as pd
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# ESG ETF 데이터 불러오기
esgu_etf = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/ESGU.csv', parse_dates=['Date'])
esgv_etf = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/ESGV.csv', parse_dates=['Date'])
vde_etf = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/VDE.csv', parse_dates=['Date'])
sdus_etf = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/SDUS.csv', parse_dates=['Date'])
tan_etf = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/TAN.csv', parse_dates=['Date'])
evus_etf = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/EVUS.csv', parse_dates=['Date'])

etf_list = [esgu_etf, esgv_etf, vde_etf, sdus_etf, tan_etf, evus_etf]

for etf in etf_list:
    etf.set_index('Date', inplace=True)

# 시장 데이터 불러오기
spy_etf = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/SPY.csv', parse_dates=['Date'])
spy_etf.set_index('Date', inplace=True)


In [273]:
# 이벤트 날짜 (가정 : 모든 ETF 동일)
event_date = pd.Timestamp("2017-01-20")

# 이벤트 윈도우 설정
estimation_window = 30  # 이벤트 이전 30일
event_window = 10  # 이벤트 전후 10일

In [274]:
df = pd.concat([esgu_etf['Close'],
                esgv_etf['Close'],
                vde_etf['Close'],
                sdus_etf['Close'],
                tan_etf['Close'],
                evus_etf['Close'],
                spy_etf['Close']],axis=1)

In [275]:
df.columns = [['esgu_etf','esgv_etf',
               'vde_etf','sdus_etf',
               'tan_etf','evus_etf',
               'spy_etf']]

In [276]:
# 윈도우 설정
estimation_start = event_date - pd.Timedelta(days=estimation_window + event_window)
estimation_end = event_date - pd.Timedelta(days=1)
event_start = event_date - pd.Timedelta(days=event_window)
event_end = event_date + pd.Timedelta(days=event_window)

# 추정 윈도우 및 이벤트 윈도우 데이터 추출
estimation_data = df.loc[estimation_start:estimation_end]
event_data = df.loc[event_start:event_end]



In [277]:
event_data["Market_Return"] = np.log(event_data["spy_etf"] / event_data["spy_etf"].shift(1))

estimation_data['Market_Return'] = estimation_data['spy_etf'].pct_change()

In [278]:
# 결과 저장
results = []


# 각 ETF별 동일한 이벤트 날짜 적용하여 분석
for etf in etf_list:
    print(f"Analyzing {etf} with SPY on event date {event_date}...")

    event_data[f"{etf}_Return"] = np.log(event_data[f"{etf}"] / event_data[f"{etf}"].shift(1))
    
    # 수익률 계산
    estimation_data[f"{etf}_Return"] = estimation_data[f'{etf}'].pct_change()
    
    # 회귀 분석 (시장 모델)
    X = sm.add_constant(estimation_data["Market_Return"])
    y = estimation_data[f"{etf}_Return"]
    model = sm.OLS(y, X).fit()
    alpha, beta = model.params

    # 비정상 수익률 (AR) 및 CAR 계산
    event_data["Expected_Return"] = alpha + beta * event_data["Market_Return"]
    a = event_data[f'{etf}_Return'].values
    b = event_data['Expected_Return'].values
        
    event_data["Abnormal_Return"] = a - b
    event_data["CAR"] = event_data["Abnormal_Return"].cumsum()

    # 결과 저장
    for date, row in event_data.iterrows():
        results.append({
            "Date": date,
            "ETF": etf,
            "Market": 'SPY',
            "Event_Date": event_date,
            "Stock_Return": row[f"{etf}_Return"],
            "Expected_Return": row["Expected_Return"],
            "Abnormal_Return": row["Abnormal_Return"],
            "CAR": row["CAR"]
        })



Analyzing                  Close        High         Low        Open  Volume  Log_Return
Date                                                                          
2016-12-06   42.745953   42.745953   42.745953   42.745953     100         NaN
2016-12-07   42.745953   42.745953   42.745953   42.745953       0    0.000000
2016-12-08   42.745953   42.745953   42.745953   42.745953       0    0.000000
2016-12-09   42.745953   42.745953   42.745953   42.745953       0    0.000000
2016-12-12   42.745953   42.745953   42.745953   42.745953       0    0.000000
...                ...         ...         ...         ...     ...         ...
2025-01-27  131.729996  131.940002  130.750000  130.750000  544300   -0.016190
2025-01-28  132.940002  133.179993  131.389999  132.149994  609400    0.009144
2025-01-29  132.320007  132.880005  131.800003  132.679993  796800   -0.004675
2025-01-30  133.009995  133.440002  132.199997  132.639999  500300    0.005201
2025-01-31  132.259995  134.119995  132.16

KeyError: '                 Close        High         Low        Open  Volume  Log_Return\nDate                                                                          \n2016-12-06   42.745953   42.745953   42.745953   42.745953     100         NaN\n2016-12-07   42.745953   42.745953   42.745953   42.745953       0    0.000000\n2016-12-08   42.745953   42.745953   42.745953   42.745953       0    0.000000\n2016-12-09   42.745953   42.745953   42.745953   42.745953       0    0.000000\n2016-12-12   42.745953   42.745953   42.745953   42.745953       0    0.000000\n...                ...         ...         ...         ...     ...         ...\n2025-01-27  131.729996  131.940002  130.750000  130.750000  544300   -0.016190\n2025-01-28  132.940002  133.179993  131.389999  132.149994  609400    0.009144\n2025-01-29  132.320007  132.880005  131.800003  132.679993  796800   -0.004675\n2025-01-30  133.009995  133.440002  132.199997  132.639999  500300    0.005201\n2025-01-31  132.259995  134.1

In [ ]:
# ✅ 결과 저장 및 시각화
results_df = pd.DataFrame(results)

import ace_tools as tools
tools.display_dataframe_to_user(name="동일 이벤트 날짜 ESG ETF 이벤트 스터디 결과", dataframe=results_df)

In [ ]:
results

[]